# DaXi alignment notebook 

## E1 - center the beam

## 1. Introduction
* talk about why we need to hold constant voltages
* talk about remember the direction of the voltages and fine-tune the galvo angles


## 2. What's included in this notebook


## 3. Relevant protocols.




# Import tools

In [2]:
import nidaqmx
import nidaqmx.system
import numpy as np
import time 

import sys
sys.path.append('/Users/xiyu.yi/Desktop/Research/Projects/P-daxi-protocol/daxi-protocol')
sys.path.append('C:/Users/PiscesScope/xiyu_workbench/daxi-protocol')
sys.path.append('C:/Users/PiscesScope/xiyu_workbench/daxi-controller/old_workbench')
from alignment_tools.actions import *
from alignment_tools.device_configs_old_daxi import *

# Adjusting galvo voltages for view 1 (1 mirror view)

In [7]:
time.sleep(0.2) # create a 2 seconds delay so we can walk to the setup and watch the beam to move.

VSG1_view1['home voltage']= 0 # 
VSG2_view1['home voltage']= 0
SG_view1['home voltage'] = 0
beta_G_view1['home voltage'] = 0 # -2 would give the correct voltage offset for beta galvo.
gamma_G_view1['home voltage'] = 0
O1_view1['home voltage'] = 0

# update the static voltages for all the galvos.
task=gohome_all([VSG1_view1, VSG2_view1, SG_view1, beta_G_view1, gamma_G_view1, O1_view1])

In [96]:
task.close()

C:\Users\PiscesScope\.conda\envs\coPylot\lib\site-packages\nidaqmx\task.py:461: ResourceWarning: Attempted to close NI-DAQmx task of name "_unnamedTask<52>" but task was already closed.
  warnings.warn(


In [10]:
time.sleep(0.2) # create a 2 seconds delay so we can walk to the setup and watch the beam to move.

VSG1_view2['home voltage']= 0 # 
VSG2_view2['home voltage']= 0
SG_view2['home voltage'] = 0 #1.075
beta_G_view2['home voltage'] = -2.537 #,-2.537  #-2.537
gamma_G_view2['home voltage'] = 0
O1_view2['home voltage'] = 0

# update the static voltages for all the galvos.
task=gohome_all([VSG1_view2, VSG2_view2, SG_view2, beta_G_view2, gamma_G_view2, O1_view2])

In [98]:
task.close()

C:\Users\PiscesScope\.conda\envs\coPylot\lib\site-packages\nidaqmx\task.py:461: ResourceWarning: Attempted to close NI-DAQmx task of name "_unnamedTask<52>" but task was already closed.
  warnings.warn(


#  Generate periodic control signal
## Needs:
1. timer
2. counter
3. define rates for the periodic control sequence.
4. define # of samples within a period.

In [5]:
# try:
# define an ao task to generate sin wave output.

# make the trigger of this ao task to be a pulse train, which is the output of a counter

# define a sampler, make it generate a pulse train, and use it as a trigger for the ao task.

# because daq cards only allow for 1 task per type for AI/O, 
# DI/O tasks if hardware timing is used. so these functions only "configures" the tasks, 
# they dont create and close tasks.

def launch_sampler(param, task):
    # try to make a pulse train as the sampling function for the ao
    task.co_channels.add_co_pulse_chan_freq(param['channel_string_I'],
                                                       idle_state=nidaqmx.constants.Level.LOW,
                                                       freq=param['sampling_rate'])
    if sampler['sample_mode'] == 'Finite':
        m=nidaqmx.constants.AcquisitionType.FINITE

    if sampler['sample_mode'] == 'Continuous':
        m=nidaqmx.constants.AcquisitionType.CONTINUOUS

    task.timing.cfg_implicit_timing(sample_mode=m,
                                    samps_per_chan=param['num_sample'])

    task.triggers.start_trigger.cfg_dig_edge_start_trig(trigger_source = param['trigger_source'],
                                                        trigger_edge = nidaqmx.constants.Slope.RISING)

    task.triggers.start_trigger.retriggerable = True
    
# and make a switch method [daq-tools].
def launch_switch(param, task, status='on'):
    "it has to be a digital channel."
    task.do_channels.add_do_chan(param['channel_string'])
    if status == 'on':
        task.write([True], auto_start=True)
    else:
        task.write([False], auto_start=True)

def reset_switch(switch_task, status='on'):
    if status == 'on':
        switch_task.write([False], auto_start=True)
        switch_task.write([True], auto_start=True)
    else:
        switch_task.write([True], auto_start=True)
        switch_task.write([False], auto_start=True)


# Prepare parameters

In [6]:
# define parameters of a swtich to be used as a trigger. (interface.)

# define parameters
sample_n = 300
x=np.linspace(0, np.pi*2, sample_n*5)
y=np.sin(x)

sw = {} # switch parameters, this line is currently labeled as DIO3 on the rack. - xiyu, 2022-09-08
# the purpose of this line is to use as a trigger for the sine v generation.
sw['instrument_name'] = "DaXi"
sw['channel I/O type'] = "DO"  #
sw['channel_string'] = "cDAQ1DIO/port0/line3"  # name of the DAQ channel that will be used to control the device, choose one.
sw['controlled_device_name'] = "switch"  # name of the deviced to be controlled. (like a switch button.)
sw['purpose'] = "this serves as a power switch to turn something on/off"  # purpose of this device.
sw['verbose'] = True  # when True, status messages will be printed.

# define sampler parameters that generates a pulsed train from an internal counter.
sampler={}
sampler['instrument_name'] = "DaXi"
sampler['channel I/O type'] = "Counter"  
sampler['channel_string_I'] = "cDAQ1/_ctr0"  
sampler['channel_string_O'] = "/cDAQ1/Ctr0InternalOutput"  
sampler['controlled_device_name'] = "sampler"  
sampler['purpose'] = "to configure a counter to generate a pulse train" 
sampler['verbose'] = True  
sampler['sampling_rate'] = sample_n
sampler['sample_mode'] = 'Finite' # specify Finite or Continuous
sampler['num_sample'] = sample_n
sampler['trigger_source'] = "/cDAQ1/PFI0" #sw['channel_string']

# define ao scan parameters 
ao_scan = {}
ao_scan['instrument_name'] = "DaXi"
ao_scan['channel I/O type'] = "AO"
ao_scan['sampler_channel_str'] = sampler['channel_string_O']
ao_scan['controlled_device_name'] = "a scanner"
ao_scan['ao_profile'] = list(y)

def get_vtrain_sine(sample_n):
    x=np.linspace(0,np.pi*2,sample_n)
    y=np.sine(x)
    return y


# Pre-launch tasks

In [7]:
sampler_task = nidaqmx.Task()
launch_sampler(sampler, task = sampler_task)

switch_task=nidaqmx.Task()
launch_switch(status = 'on', param = sw, task = switch_task)

# Launch tasks

In [8]:
sampler_task.start()

# Trigger tasks


In [9]:
# setup the scanners.
reset_switch(status='on', switch_task=switch_task)

# Close tasks

In [10]:
sampler_task.close()
switch_task.close()

In [12]:
action_params=ao_scan
device_params=SG

task_ao_scanner = nidaqmx.Task()
task_ao_scanner.ao_channels.add_ao_voltage_chan(device_params['channel_string'])  # scanning galvo
task_ao_scanner.timing.cfg_samp_clk_timing(rate=sample_n,
                                   source=action_params['sampler_channel_str'],
                                   sample_mode=nidaqmx.constants.AcquisitionType.CONTINUOUS)
task_ao_scanner.start()

DaqError: Generation cannot be started because the output buffer is empty.
Write data before starting a buffered generation. The following actions can empty the buffer: changing the size of the buffer, unreserving a task, setting the Regeneration Mode property, changing the Sample Mode, or configuring retriggering.
Task Name: _unnamedTask<2>

Status Code: -200462

In [8]:
task_ao_scanner.stop()
task_ao_scanner.close()

In [48]:
gohome(SG)

Now setting [scanning galvo] to the home position [voltage = 0v] with [offset = 0v,False] through channel ['cDAQ1AO/ao0']
Task perfromed successfully.
Task closed.


In [ ]:
# setup a counter task to pair with the controller task.
#? what does counter do?
counter = nidaqmx.Task("counter0")
counter.close()

In [51]:
counter.close()
task.close()

C:\Users\PiscesScope\.conda\envs\coPylot\lib\site-packages\nidaqmx\task.py:457: ResourceWarning: Attempted to close NI-DAQmx task of name "counter0" but task was already closed.
  warnings.warn(


In [ ]:
    with nidaqmx.Task() as task:
        task.ao_channels.add_ao_voltage_chan(ch)
        task.write([value], auto_start=True)

In [62]:
time.sleep(2)

In [38]:
def view_switching_galvo1(channel="cDAQ1AO/ao1", voltage=4.2):
    # create task to set the channel to the desired voltage.
    
    
    
    

32

In [18]:
counter_names = [ci.name for ci in DAQ_device.ci_physical_chans]
print(counter_names)

['cDAQ1DIO/ctr0', 'cDAQ1DIO/ctr1', 'cDAQ1DIO/ctr2', 'cDAQ1DIO/ctr3']


In [19]:
print([co.name for co in DAQ_device.co_physical_chans])

['cDAQ1DIO/ctr0', 'cDAQ1DIO/ctr1', 'cDAQ1DIO/ctr2', 'cDAQ1DIO/ctr3', 'cDAQ1DIO/freqout']


In [22]:
DAQ_device.co_max_size

32

In [26]:
DAQ_device.di_lines.channel_names

['cDAQ1DIO/port0/line0',
 'cDAQ1DIO/port0/line1',
 'cDAQ1DIO/port0/line2',
 'cDAQ1DIO/port0/line3',
 'cDAQ1DIO/port0/line4',
 'cDAQ1DIO/port0/line5',
 'cDAQ1DIO/port0/line6',
 'cDAQ1DIO/port0/line7']

In [27]:
DAQ_device.do_lines.channel_names

['cDAQ1DIO/port0/line0',
 'cDAQ1DIO/port0/line1',
 'cDAQ1DIO/port0/line2',
 'cDAQ1DIO/port0/line3',
 'cDAQ1DIO/port0/line4',
 'cDAQ1DIO/port0/line5',
 'cDAQ1DIO/port0/line6',
 'cDAQ1DIO/port0/line7']

In [28]:
DAQ_device.name

'cDAQ1DIO'

In [7]:
# try counter.

In [4]:
# set up the counter to retrigger the ao and do channels
task_ctr_retrig = nidaqmx.Task()
task_ctr_retrig.co_channels.add_co_pulse_chan_freq(
    "cDAQ1/_ctr0",
    idle_state=nidaqmx.constants.Level.LOW,
    freq=200,
)
task_ctr_retrig.timing.cfg_implicit_timing(
    sample_mode=nidaqmx.constants.AcquisitionType.FINITE,
    samps_per_chan=200,
)
task_ctr_retrig.triggers.start_trigger.cfg_dig_edge_start_trig(
    trigger_source=sw['channel_string'], trigger_edge=nidaqmx.constants.Slope.RISING
)
task_ctr_retrig.triggers.start_trigger.retriggerable = True



In [5]:
# set up ao channel
action_params=ao_scan
device_params=SG

task_ao_scanner = nidaqmx.Task()
task_ao_scanner.ao_channels.add_ao_voltage_chan(device_params['channel_string'])  # scanning galvo
task_ao_scanner.timing.cfg_samp_clk_timing(rate=sample_n,
                                   source=action_params['sampler_channel_str'],
                                   sample_mode=nidaqmx.constants.AcquisitionType.CONTINUOUS)
task_ao_scanner.start()




DaqError: Generation cannot be started because the output buffer is empty.
Write data before starting a buffered generation. The following actions can empty the buffer: changing the size of the buffer, unreserving a task, setting the Regeneration Mode property, changing the Sample Mode, or configuring retriggering.
Task Name: _unnamedTask<1>

Status Code: -200462

In [ ]:
task_ctr_retrig.start()
for _ in range(self.nb_timepoints):
    for v in range(len(data_ao)):  # change view
        task_ao.write(data_ao[v])
        task_ao.start()
        print("number of slices to acquire:", self.nb_slices)
        for ch in range(
            nr_channels
        ):
            # regenerate do date for non-interleaved multichannel acquisition
            if not interleave and nr_channels > 1:
                data_do = self._get_do_data(
                    nr_channels, interleave, current_ch=ch
                )

            task_do.write(data_do)
            task_do.start()

            task_ctr_loop.start()
            counts = 0
            while (
                counts < self.nb_slices + 1
            ):  # Flash4.0 outputs 1 more pulse than asked
                counts = task_ctr_loop.read()
                time.sleep(0.005)  # wait time during loop
            task_ctr_loop.stop()
            print("counts: ", counts)
            # print("counts: ", counts)
            time.sleep(
                self.exposure + 0.1
            )  # add time to allow ao and do output for the last frame
            task_do.stop()
            print("one stack done!")
        task_ao.stop()

# task_ctr_retrig.stop()
task_ctr_retrig.close()
task_ctr_loop.close()